In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random
from scipy.spatial import distance_matrix

In [2]:
# Useful functions

def Thresh(x):
    x = np.array(x)
    u = (x>0)*2-1
    return u


def Error(x, y):

    d = []
    for xx, yy in zip(x,y):
        dd = 0.
        for xxx,yyy in zip(xx,yy):
            if xxx==1 and yyy!=1:
                dd += 1.
            elif yyy==1 and xxx!=1:
                dd += 1.
        d.append(dd)
    return d


def Perturb(x, p):
    
    y = np.copy(x)
    for yy in y:
        for k in range(len(yy)):
            if np.random.rand()<p:
                yy[k] = random.choice([1,-1])
    return y


def Update(W, x, b):
    xnew = x @ W - b
    return Thresh(xnew)



def make_lattice(N):    

    spin = [-1,1]
    lattice = np.random.choice(spin,(N,N))
    return lattice


def lattice_points(N,s):

    l = ((N+1)/2-0.5)*s

    lattice_points = []

    for x in np.arange(-l+0.5,l+0.5,s):

        for y in np.arange(l-0.5,-l,-s):

            lattice_points.append([x,y]) 

    xs = [i[0] for i in lattice_points]
    ys = [i[1] for i in lattice_points]
    
       
    rij=[]
    
    for i in range(N**2):
        for j in range(N**2):
        
            rij.append([ys[i]-ys[j],xs[i]-xs[j]])
    points = np.array(rij)
        
    vector_rij=points.reshape(N**2,N**2,2)
    
    return xs,ys,vector_rij,lattice_points



def distance(N):
    
    for i in range(N**2):
        for j in range(N**2):
            dists = distance_matrix(lattice_points,lattice_points)
      
    return dists



def Spin_vectors(N):
    
    
    x_direction=[]
    y_direction=[]  
    
    
    %matplotlib qt
    
    for i in range(N):
        if i%2==1:
            for j in range(N):
                #angle1=random.uniform(0, 4*np.pi)
                angle1=0
                if j%2!=1:
                    x_direction.append(np.cos(angle1))
                    y_direction.append(np.sin(angle1))
                else:
                    x_direction.append(np.cos(angle1))
                    y_direction.append(np.sin(angle1))

        else:

            for j in range(N):
                #angle2=random.uniform(0, 4*np.pi)
                angle2=0
                if j%2==1:
                    x_direction.append(np.cos(angle2))
                    y_direction.append(np.sin(angle2))
                else:
                    x_direction.append(np.cos(angle2))
                    y_direction.append(np.sin(angle2))

    spin_configuration=[]
        
    for i in range(N**2):
        c=[]
        c.append(x_direction[i])
        c.append(y_direction[i])
        spin_configuration.append(c)
        
  
    return x_direction,y_direction,spin_configuration


def Jij(N):
    
    J=np.zeros((N**2,N**2))
    const=1
    for i in range(N**2):
        for j in range(N**2):
            
            if i == j:
                continue
                
            if dists[i][j]!=0:
                #J[i][j]=1
                J[i][j]=0.5*const*dists[i][j]**(-3)*np.dot(spin_configuration[i],spin_configuration[j])-0.5*const*3*dists[i][j]**(-5)*np.dot(spin_configuration[i],vector_rij[i][j])*np.dot(spin_configuration[j],vector_rij[i][j])
    
    return J



def average_energy(J,lattice):
    
    
    energy = 0
    n = len(lattice)
    for i in range(n):
        for j in range(n):
            energy += -J[i,j]*lattice[i,j]*((lattice[(i+1)%n,j])+(lattice[(i-1),j])+(lattice[i,(j+1)%n])+(lattice[i,(j-1)]))
    average_energy = 0.5*energy/(n**2)
    return average_energy


def spin_inversion(J,lattice, tempreture):
    
    
    k=1
    n = len(lattice)
    i,j = np.random.randint(n),np.random.randint(n)
    
    delta_energy = -2*(-J[i][j]*lattice[i,j]*((lattice[(i+1)%n,j])+(lattice[(i-1),j])+(lattice[i,(j+1)%n])+(lattice[i,(j-1)])))
    
    if delta_energy < 0 or (np.random.rand() < np.exp(-delta_energy/(k*tempreture))):
        
        lattice[i,j] = - lattice[i,j]
             
    return lattice
    

In [3]:
s = 1
N = 6 

ensembles = 50


    
xs,ys,vector_rij,lattice_points = lattice_points(N,s)
dists = distance(N)    
x_direction,y_direction,spin_configuration=Spin_vectors(N)

J=Jij(N)

'energy and tempurature'
tempreture = np.linspace(1,5,40)           
en=[]
for temp in tempreture:
    ae=0
    for ensemble in range(ensembles):
        
        new_lattice = make_lattice(N) 
        for rep in range(N**4):
            new_lattice = spin_inversion(J,new_lattice,temp)
        ae+=average_energy(J,new_lattice)
    en.append(ae/ensembles)

        


fig, ax = plt.subplots()

ax.scatter(tempreture,en,s=10)
plt.show()


'''energy and ensembles'''
# for ensemble in range(ensembles):
#         new_lattice = make_lattice(N)
#         ae=[]
#         for rep in range(N**2):
#             new_lattice = spin_inversion(J,new_lattice,0)
#             ae.append(average_energy(J,new_lattice))
        

           
# x = np.linspace(0,ensembles,len(ae))   

# fig, ax = plt.subplots()
# ax.plot(x,ae, linewidth=2.0)
# ax.scatter(x,ae,s=10)
# plt.show()

'energy and ensembles'

In [4]:
X=np.copy(new_lattice) 
b = np.zeros((1,N))
b = np.sum(X, axis=0) / N
W = ( X.T @ X ) / N - np.eye(N)

In [5]:
# Perturbed memory
k = np.random.randint(len(X))
Y = Perturb(X, p=0.2)
x = Y[k:k+1]
err = Error(x, X[k:k+1])


In [6]:
#Iterat network to steady state
n_iters = 10
for k in range(n_iters):
    #print(x)
    x_new = Update(W, x, b)
    print(Error(x, x_new))
    x = x_new

[2.0]
[0.0]
[0.0]
[0.0]
[0.0]
[0.0]
[0.0]
[0.0]
[0.0]
[0.0]


In [7]:
#How close are we to recalling a memory?
for idx,t in enumerate(X):
    d = Error(x, [t])
    print('Memory '+str(idx)+' has error '+str(d))

Memory 0 has error [2.0]
Memory 1 has error [2.0]
Memory 2 has error [0.0]
Memory 3 has error [6.0]
Memory 4 has error [0.0]
Memory 5 has error [5.0]
